# Google Trends
Google Trends is very interesting as one can get the interest for a particuliar word or sentence on the internet. One extra training value that our model could benefit from is the interest on google. Therefore let's call the google trends api and see if we can get this using the primary title.

In [1]:
import pandas as pd
from script import DataManager

dm = DataManager()

In [2]:
dm.train_df.sort_values(by = ["numVotes"],ascending = False)[:10]

,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,numVotes,label,special_chars
2974,tt0111161,The Shawshank Redemption,The Shawshank Redemption,1994,142,2503641.0,True,
5039,tt0468569,The Dark Knight,The Dark Knight,2008,152,2453191.0,True,
3449,tt0137523,Fight Club,Fight Club,1999,139,1969585.0,True,
2916,tt0109830,Forrest Gump,Forrest Gump,1994,142,1932502.0,True,
3432,tt0133093,The Matrix,The Matrix,1999,136,1787064.0,True,
3589,tt0167260,The Lord of the Rings: The Return of the King,nan,2003,201,1729584.0,True,
1413,tt0068646,The Godfather,nan,1972,175,1725806.0,True,o
5334,tt0816692,Interstellar,Interstellar,2014,169,1651772.0,True,
7073,tt1853728,Django Unchained,Django Unchained,2012,165,1448177.0,True,
2685,tt0102926,The Silence of the Lambs,nan,1991,118,1348539.0,True,


Let's try for **The Dark Knight** and **Fight Club** as both are amazing movies and also for **Scavengers** as this seems to be a very unpopular movie from 2013.

In [3]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [4]:
from pytrends.request import TrendReq
import time
pytrend = TrendReq()
trends=dict()

#List of title to compare (should be all movies in our dataset)
#kw_list = ["The Dark Knight","Fight Club","Scavengers"]
kw_list = dm.train_df["primaryTitle"].to_list()
print(len(kw_list))
kw_list.remove("The Shawshank Redemption")
print(len(kw_list))
kw_pre_compare = list(chunks(kw_list,4))
kw_to_compare = [x+ ["The Shawshank Redemption"] for x in kw_pre_compare]
kw_to_compare

7959
7958


[['Mickey',
  'The Doll',
  'The Mark of Zorro',
  "The Parson's Widow",
  'The Shawshank Redemption'],
 ['Way Down East',
  'The Kid',
  'Destiny',
  'Orphans of the Storm',
  'The Shawshank Redemption'],
 ['Haxan',
  'Robin Hood',
  'The Faithful Heart',
  'The Saga of Gosta Berling',
  'The Shawshank Redemption'],
 ['The Hunchback of Notre Dame',
  'Our Hospitality',
  'The Pilgrim',
  'Three Ages',
  'The Shawshank Redemption'],
 ['Why Worry?',
  'Girl Shy',
  'The Last Laugh',
  'The Navigator',
  'The Shawshank Redemption'],
 ['Peter Pan',
  'Strike',
  'The Thief of Bagdad',
  'The Freshman',
  'The Shawshank Redemption'],
 ['The Joyless Street',
  'The Gold Rush',
  'Greed',
  'The Little Colonel',
  'The Shawshank Redemption'],
 ['The Phantom of the Opera',
  'The Wizard of Oz',
  'Battling Butler',
  'Ben-Hur: A Tale of the Christ',
  'The Shawshank Redemption'],
 ['Faust',
  'A Page of Madness',
  'By the Law',
  'The Scarlet Letter',
  'The Shawshank Redemption'],
 ['3 Bad 

In [5]:
from pytrends.request import TrendReq
from os.path import exists
import time
pytrend = TrendReq()
trends=dict()

j = 0

for i in kw_to_compare:
    if exists("google_trends_movies/"+str(j)+"_google_trends.csv") == False:
        pytrend.build_payload(i,cat=34,timeframe='2004-01-01 2022-01-01')
        trends = pytrend.interest_over_time()
        trends.to_csv("google_trends_movies/"+str(j)+"_google_trends.csv")
        print(i)
        print(j)
        time.sleep(5)
    j+=1

trends

['Holiday Rush', 'False Positive', 'Slumdog Millionaire', 'Thunder Force', 'The Shawshank Redemption']
1097


ResponseError: The request failed: Google returned a response with code 429.

The google trend api has several problems. Both can be overcome but before starting it's worth discussing if we think that provides real value.

**1)** We can only query and compare 5 items at a time. That means that to get the information for every  movie we would need to loop through our dataframe **while** keeping always one item the same for all comparison because of the following.

**2)** The comparison is always relative. So even if I do one query that compares *The Dark Knight* to *Fight Club* and ones that compares *The Dark Knight* to *Scavengers* the value for Dark Knight won't be the same in both cases. That's not the end of the world because we can normalize and average the results but this might be a hustle for 2000 separate dataframes

In [ ]:
historicaldf = pytrend.get_historical_interest(kw_list, year_start=2020, month_start=10, day_start=1, hour_start=0, year_end=2021, month_end=10, day_end=1, hour_end=0, cat=0, geo='', gprop='', sleep=0)
#visualise
#plot a timeseries chart
historicaldf.plot(figsize=(20, 12))

#plot seperate graphs, using theprovided keywords
historicaldf.plot(subplots=True, figsize=(20, 12))